## Implementación del control de versiones con Amazon DynamoDB
Algunas aplicaciones requieren que registre cambios en los datos a lo largo del tiempo, y puede identificar estos cambios mediante una marca de tiempo o un número para que sea posible recuperar versiones específicas más adelante. Además, debes poder recuperar fácilmente la versión más reciente y el diseño de la aplicación para mantener la integridad de los datos a medida que las versiones aumentan constantemente.

En esta publicación se explica cómo diseñar e implementar el control de versiones basado en tiempo y número en Amazon DynamoDB. En los cuatro ejemplos se utiliza una clave principal compuesta para modelar las versiones históricas de los datos y facilitar la recuperación de la versión más reciente de los datos. 

### Versionado basado en el tiempo
Cuando los cambios en los datos se producen de forma secuencial a lo largo del tiempo, puedes utilizar una marca de tiempo para versionar los datos siempre que acompañe a los datos modificados. Por ejemplo, quizás necesites diseñar un modelo de datos para una fábrica que tiene muchas partes, y cada parte envía datos de estado cada 2 minutos. Tienes que almacenar el estado histórico de cada parte en la base de datos. El siguiente modelo de datos ilustra cómo podría modelar estos datos en DynamoDB.

![image](https://d2908q01vomqb2.cloudfront.net/887309d048beef83ad3eabf2a79a64a389ab1c9f/2020/12/15/DBBLOG-1276-1.jpg)

Cada dato de estado (1) se añade a la colección de elementos de equipo, y la clave de clasificación contiene la marca de tiempo acompañada de los datos de estado. El elemento Metadatos (2) actúa como metadatos para una entidad de equipo, conteniendo atributos específicos de esta entidad, como Nombre, FactoryID y LineID. Este elemento rara vez se actualiza.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [5]:
data = [
    {'PK': 'Equipment#1', 'SK': '2023-10-03T12:32:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#1', 'SK': '2023-11-05T12:12:00', 'State': 'WARNING1'}, 
    {'PK': 'Equipment#1', 'SK': '2023-11-06T12:05:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#6', 'SK': '2024-03-07T22:09:29', 'State': 'ERROR'},
    {'PK': 'Equipment#6', 'SK': '2024-03-30T22:09:29', 'State': 'WARNING2'},
    {'PK': 'Equipment#118', 'SK': '2023-12-15T08:30:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#118', 'SK': '2023-12-16T09:45:00', 'State': 'WARNING1'},
    {'PK': 'Equipment#118', 'SK': '2023-12-17T10:20:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#118', 'SK': '2023-12-18T11:05:00', 'State': 'ERROR'},
    {'PK': 'Equipment#118', 'SK': '2023-12-19T12:15:00', 'State': 'WARNING2'}
]

equipment1 = {'PK': 'Equipment#1', 'SK': 'Metadata', 'Name': 'Equipment-001', 'FactoryId': 'F#88546'}
equipment6 = {'PK': 'Equipment#6', 'SK': 'Metadata', 'Name': 'Equipment-006', 'FactoryId': 'F#56658'}
equipment118 = {'PK': 'Equipment#118', 'SK': 'Metadata', 'Name': 'Equipment-118', 'FactoryId': 'F#88985'}

In [2]:
dt = DynamoTable(
    table_name='VersionTable'
    #profile_name='my-profile'
)

if not dt.table_name:
    dt.create_table(
        table_name='VersionTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S'
        #profile_name='my-profile'
    )
    
else:
    print(dt)

Table info:
 - Table name: VersionTable
 - Table arn: arn:aws:dynamodb:us-east-1:089715336747:table/VersionTable
 - Table creation: 2024-08-20T13:43:27
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]
 - Attribute definitions: [{'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]
 - Table class: STANDARD
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF



In [6]:
dt.add_item(equipment1)
dt.add_item(equipment6)
dt.add_item(equipment118)
dt.write_batch(data)

Data loaded successfully in 0.14 seconds.


El tipo de datos del atributo sort key es String, lo que significa que los elementos de una colección de elementos se ordenan por orden de bytes UTF-8. Cuando se ejecuta una operación de consulta, los resultados se devuelven por defecto en orden ascendente. En el código anterior, para invertir el orden, el parámetro `ScanIndexForward` se establece en false, lo que significa que el elemento con Metadata como clave de ordenación es el primer elemento devuelto, seguido de los elementos con timestamp como clave de ordenación, del más reciente al más antiguo. Para recuperar sólo el elemento con la fecha más reciente en la clave de ordenación, se utiliza la función `begins_with()` en la `KeyConditionExpression`, así como `Limit`, que especifica el número máximo de elementos a evaluar.

In [7]:
result = dt.query(pk_value="Equipment#118", sk_value="2023-12*", to_pandas=True, reverse=False)
result

,State,PK,SK
0,WARNING2,Equipment#118,2023-12-19T12:15:00
1,ERROR,Equipment#118,2023-12-18T11:05:00
2,NORMAL,Equipment#118,2023-12-17T10:20:00
3,WARNING1,Equipment#118,2023-12-16T09:45:00
4,NORMAL,Equipment#118,2023-12-15T08:30:00


In [8]:
result = dt.query(pk_value="Equipment#6", to_pandas=True)
result

,State,PK,SK,FactoryId,Name
0,ERROR,Equipment#6,2024-03-07T22:09:29,NaN,NaN
1,WARNING2,Equipment#6,2024-03-30T22:09:29,NaN,NaN
2,NaN,Equipment#6,Metadata,F#56658,Equipment-006


### Versión numérica
Por el contrario, algunas aplicaciones requieren una versión basada en números que se actualice a la versión inmediatamente superior después de cada cambio en los datos, incluso aunque se disponga de una marca de tiempo. Para modelar versiones históricas de datos y recuperar fácilmente la versión más reciente de los datos, puede utilizar una clave primaria compuesta para su tabla DynamoDB y seguir el patrón de diseño de control de versiones utilizando prefijos de clave de ordenación.

Supongamos que necesita utilizar una versión basada en números en lugar de una versión basada en tiempo para nuestro caso de uso de fábrica, en el que debe almacenar el estado histórico de cada equipo en la base de datos. El siguiente modelo de datos ilustra cómo puede modelar estos datos en DynamoDB.

In [10]:
dt = DynamoTable(
    table_name='VersionNumTable'
    #profile_name='my-profile'
)

if not dt.table_name:
    dt.create_table(
        table_name='VersionNumTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S'
        #profile_name='my-profile'
    )
    
print(dt)

Table info:
 - Table name: VersionNumTable
 - Table arn: arn:aws:dynamodb:us-east-1:089715336747:table/VersionNumTable
 - Table creation: 2024-08-20T13:45:33
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]
 - Attribute definitions: [{'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]
 - Table class: STANDARD
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF



In [11]:
data = [
    {'PK': 'Equipment#1', 'SK': 'v01', 'Time': '2023-10-03T12:32:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#1', 'SK': 'v02', 'Time': '2023-11-05T12:12:00', 'State': 'WARNING1'}, 
    {'PK': 'Equipment#1', 'SK': 'v03', 'Time': '2023-11-06T12:05:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#6', 'SK': 'v01', 'Time': '2024-03-07T22:09:29', 'State': 'ERROR'},
    {'PK': 'Equipment#6', 'SK': 'v02', 'Time': '2024-03-30T22:09:29', 'State': 'WARNING2'},
    {'PK': 'Equipment#118', 'SK': 'v01', 'Time': '2023-12-15T08:30:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#118', 'SK': 'v02', 'Time': '2023-12-16T09:45:00', 'State': 'WARNING1'},
    {'PK': 'Equipment#118', 'SK': 'v03', 'Time': '2023-12-17T10:20:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#118', 'SK': 'v04', 'Time': '2023-12-18T11:05:00', 'State': 'ERROR'},
    {'PK': 'Equipment#118', 'SK': 'v05', 'Time': '2023-12-19T12:15:00', 'State': 'WARNING2'}
]

equipment1 = {'PK': 'Equipment#1', 'SK': 'Metadata', 'Name': 'Equipment-001', 'FactoryId': 'F#88546'}
equipment6 = {'PK': 'Equipment#6', 'SK': 'Metadata', 'Name': 'Equipment-006', 'FactoryId': 'F#56658'}
equipment118 = {'PK': 'Equipment#118', 'SK': 'Metadata', 'Name': 'Equipment-118', 'FactoryId': 'F#88985'}

In [12]:
dt.add_item(equipment1)
dt.add_item(equipment6)
dt.add_item(equipment118)
dt.write_batch(data)

Data loaded successfully in 0.14 seconds.


![image](https://d2908q01vomqb2.cloudfront.net/887309d048beef83ad3eabf2a79a64a389ab1c9f/2020/12/15/DBBLOG-1276-2.jpg)

El elemento con la clave de clasificación Metadatos (1) actúa como metadatos para una entidad de equipo, conteniendo atributos específicos de la entidad, como Nombre, FactoryID y LineID. Este elemento rara vez se actualiza.

El elemento con v0 en la clave de clasificación (2) contiene una copia de la última revisión del equipo, incluyendo todos sus atributos, y también el atributo Latest, que contiene el número de la última versión. Este elemento siempre se actualiza con el contenido de la versión más reciente.

Cada vez que se actualiza el equipo, se añade un nuevo elemento con la versión inmediatamente superior en la clave de clasificación y el contenido actualizado (3).

In [13]:
data_v0 = [
    {'PK': 'Equipment#1', 'SK': 'v00', 'Time': '2023-11-06T12:05:00', 'State': 'NORMAL'},
    {'PK': 'Equipment#6', 'SK': 'v00', 'Time': '2024-03-30T22:09:29', 'State': 'WARNING2'},
    {'PK': 'Equipment#118', 'SK': 'v00', 'Time': '2023-12-19T12:15:00', 'State': 'WARNING2'}
]
dt.write_batch(data_v0)

Data loaded successfully in 0.14 seconds.


In [17]:
dt.query(pk_value="Equipment#118", sk_value="v*", reverse=False, limit=1)

[{'SK': 'v05',
  'Time': '2023-12-19T12:15:00',
  'PK': 'Equipment#118',
  'State': 'WARNING2'}]

In [18]:
dt.query(pk_value="Equipment#118", sk_value="v05", consumed_capacity=True)

Consumed Capacity: 0.5


{'SK': 'v05',
 'Time': '2023-12-19T12:15:00',
 'PK': 'Equipment#118',
 'State': 'WARNING2'}

### Conclusión

Al diseñar una aplicación, los requisitos que debe satisfacer determinan si debe elegir una versión basada en tiempo o en números. Cuando se utiliza una clave primaria compuesta para una tabla en DynamoDB, una colección de elementos de entidad se modela de forma que se pueda recuperar fácilmente la versión más reciente y, al mismo tiempo, mantener las revisiones históricas. Cuando diseñe e implemente un caso de uso que requiera el control de versiones, tenga en cuenta la frecuencia con la que se añaden nuevas revisiones y la tolerancia de su caso de uso a las versiones incoherentes.